In [1]:
# Copyright (c) OpenMMLab. All rights reserved.
import argparse
import os
import os.path as osp
import warnings
from copy import deepcopy

from mmengine import ConfigDict
from mmengine.config import Config, DictAction
from mmengine.runner import Runner

from mmdet.engine.hooks.utils import trigger_visualization_hook
from mmdet.evaluation import DumpDetResults
from mmdet.registry import RUNNERS
from mmdet.utils import setup_cache_size_limit_of_dynamo

In [2]:
class DotDict(dict):
    """
    A dictionary with dot notation access to keys.
    """
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [3]:
def parse_args():
    args = DotDict()
    
    args['config'] = "/home/dmsai2/mmdetection/my_configs/faster-renn_r101_fpn_1x_coco.py" # train config file path
    args['checkpoint'] = "/home/dmsai2/mmdetection/work_dir/epoch_48.pth"
    args['work_dir'] = "/home/dmsai2/mmdetection/work_dir/" # the dir to save logs and models
    args['out'] = "/home/dmsai2/mmdetection/work_dir/pkls/resnet101_fastrcnn_epoch48.pkl" # dump predictions to a pickle file for offline evaluation
    args['show'] = True # show prediction results
    args['show_dir'] = "/home/dmsai2/mmdetection/show_dir/" # directory where painted images will be saved
    args['wait_time'] = 2 # the interval of show (s)
    args['launcher'] = "none" # job launcher
    args['tta'] = False
    args['local_rank'] = 0
    
    if 'LOCAL_RANK' not in os.environ:
        os.environ['LOCAL_RANK'] = str(args['local_rank'])
        
    return args

In [4]:
args = parse_args()

In [5]:
setup_cache_size_limit_of_dynamo()

In [6]:
# load config
cfg = Config.fromfile(args['config'])
cfg.launcher = args['launcher']
# if args['cfg_options'] is not None:
#     cfg.merge_from_dict(args['cfg_options'])

# work_dir is determined in this priority: CLI > segment in file > filename
if args['work_dir'] is not None:
    # update configs according to CLI args if args.work_dir is not None
    cfg.work_dir = args['work_dir']
elif cfg.get('work_dir', None) is None:
    # use config filename as default work_dir if cfg.work_dir is None
    cfg.work_dir = osp.join('./work_dirs',
                            osp.splitext(osp.basename(args['config']))[0])

In [7]:
cfg.load_from = args['checkpoint']

In [8]:
if args['show'] or args['show_dir']:
    cfg = trigger_visualization_hook(cfg, args)

In [9]:
if args.tta:
    if 'tta_model' not in cfg:
        warnings.warn('Cannot find ``tta_model`` in config, '
                        'we will set it as default.')
        cfg.tta_model = dict(
            type='DetTTAModel',
            tta_cfg=dict(
                nms=dict(type='nms', iou_threshold=0.5), max_per_img=100))
    if 'tta_pipeline' not in cfg:
        warnings.warn('Cannot find ``tta_pipeline`` in config, '
                        'we will set it as default.')
        test_data_cfg = cfg.test_dataloader.dataset
        while 'dataset' in test_data_cfg:
            test_data_cfg = test_data_cfg['dataset']
        cfg.tta_pipeline = deepcopy(test_data_cfg.pipeline)
        flip_tta = dict(
            type='TestTimeAug',
            transforms=[
                [
                    dict(type='RandomFlip', prob=1.),
                    dict(type='RandomFlip', prob=0.)
                ],
                [
                    dict(
                        type='PackDetInputs',
                        meta_keys=('img_id', 'img_path', 'ori_shape',
                                    'img_shape', 'scale_factor', 'flip',
                                    'flip_direction'))
                ],
            ])
        cfg.tta_pipeline[-1] = flip_tta
    cfg.model = ConfigDict(**cfg.tta_model, module=cfg.model)
    cfg.test_dataloader.dataset.pipeline = cfg.tta_pipeline

In [10]:
# build the runner from config
if 'runner_type' not in cfg:
    # build the default runner
    runner = Runner.from_cfg(cfg)
else:
    # build customized runner from the registry
    # if 'runner_type' is set in the cfg
    runner = RUNNERS.build(cfg)


06/11 00:56:13 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.19 (default, Mar 20 2024, 19:58:24) [GCC 11.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 479327661
    GPU 0: NVIDIA GeForce RTX 3070
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.3.0+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.6 (Git Hash 86e6af5974177e513fd3fee58425e1063e7f1361)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_5

/home/dmsai2/mmdetection/mmdet/engine/hooks/visualization_hook.py:68: UserWarning: The show is True, it means that only the prediction results are visualized without storing data, so vis_backends needs to be excluded.
  warnings.warn('The show is True, it means that only '


In [11]:
# add `DumpResults` dummy metric
if args.out is not None:
    assert args.out.endswith(('.pkl', '.pickle')), \
        'The dump file must be a pkl file.'
    runner.test_evaluator.metrics.append(
        DumpDetResults(out_file_path=args.out))

loading annotations into memory...
Done (t=0.16s)
creating index...
index created!
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
06/11 00:56:15 - mmengine - WARNING - The prefix is not set in metric class DumpDetResults.


In [13]:
# start testing
runner.test()

Loads checkpoint by local backend from path: /home/dmsai2/mmdetection/work_dir/epoch_48.pth
06/11 00:56:37 - mmengine - INFO - Load checkpoint from /home/dmsai2/mmdetection/work_dir/epoch_48.pth


/home/dmsai2/.conda/envs/openmmlab/lib/python3.8/site-packages/mmengine/visualization/visualizer.py:760: UserWarning: Warning: The bbox is out of bounds, the drawn bbox may not be in the image
  warnings.warn(
/home/dmsai2/.conda/envs/openmmlab/lib/python3.8/site-packages/mmengine/visualization/visualizer.py:831: UserWarning: Warning: The polygon is out of bounds, the drawn polygon may not be in the image
  warnings.warn(


06/11 00:59:47 - mmengine - INFO - Epoch(test) [ 50/125]    eta: 0:04:35  time: 3.6783  data_time: 3.6357  memory: 443  


/home/dmsai2/.conda/envs/openmmlab/lib/python3.8/site-packages/mmengine/visualization/visualizer.py:508: UserWarning: Warning: The text is out of bounds, the drawn text may not be in the image
  warnings.warn(


06/11 01:01:35 - mmengine - INFO - Epoch(test) [100/125]    eta: 0:01:14  time: 2.2696  data_time: 2.2392  memory: 443  
06/11 01:01:44 - mmengine - INFO - Evaluating bbox...
Loading and preparing results...


AssertionError: Results do not correspond to current coco set